In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Сиды

In [ ]:
import numpy as np
import random
import os 

seed=42

os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
random.seed(seed)


# Импорты

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import pandas as pd
# ВНИМАНИЕ ВНИМАНИЕ ПРЕДИКТ НА МЕТРИКУ: f1_score 
from sklearn.metrics import f1_score
from catboost import Pool, CatBoostClassifier, CatBoostRegressor
import matplotlib.pyplot as plt 


# TEST MODE

In [ ]:
TEST_MODE=True

# Загрузка данных

In [ ]:
train=pd.read_csv('')
test=pd.read_csv('')
sample=pd.read_csv('')

#smth else?


In [ ]:
id_test=test['id']

# cat and num feats 

In [ ]:
drop_nonsence=['']

In [ ]:
train=train.drop(columns=drop_nonsence)
test=test.drop(columns=drop_nonsence)

In [ ]:
cat_feats=train.select_dtypes(include='object').columns.tolist()
num_feats=train.drop(columns='target').select_dtypes(include='number').columns.tolist()


# If TEST MODE

In [ ]:
if TEST_MODE:
    train=train[:100]
else:
    train=train

# EDA

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
for col in num_feats:
    plt.figure()
    plt.hist(train[col])
    
    plt.xlabel(col)
    plt.ylabel('Частота')
    plt.show()

In [ ]:
for col in num_feats:
    plt.figure()
    plt.hist(test[col])
    
    plt.xlabel(col)
    plt.ylabel('Частота')
    plt.show()

In [ ]:
#for col in num_feats:
correlation=train[num_feats].corrwith(train['target'])
plt.figure()
correlation.plot(kind='bar')
plt.show()

In [ ]:
for col in cat_feats:
    counts=train[col].value_counts()
    plt.figure()
    counts.plot(kind='bar')
    plt.xlabel(col)
    plt.ylabel('Частота')

In [ ]:
for col in cat_feats:
    cat_crt=pd.crosstab(train['col'], train['target'], normalize='columns')*100
    ax=cat_crt.plot(kind='bar')
    ax.set_xlabel(col)
    ax.set_ylabel('%')
    plt.show()

# Cleaning???

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def _iqr_bounds(s: pd.Series, whisker: float = 1.5):
    s = s.dropna()
    if s.empty:
        return np.nan, np.nan
    q1 = s.quantile(0.25, interpolation="linear")
    q3 = s.quantile(0.75, interpolation="linear")
    iqr = q3 - q1
    low = q1 - whisker * iqr
    high = q3 + whisker * iqr
    return low, high

def boxplot_clean(
    df: pd.DataFrame,
    num_feats,
    by=None,                       # str or list[str] for group-wise bounds, or None for global
    mode: str = "remove",          # "remove" = drop outlier rows, "cap" = winsorize to bounds
    whisker: float = 1.5,
    max_remove_frac: float = 0.10, # if > this would be removed, auto-cap instead
    min_group_size: int = 20       # for small groups, fallback to global bounds
):
    out = df.copy()
    records = []

    if by is None:
        # Global IQR per feature
        for col in num_feats:
            low, high = _iqr_bounds(out[col], whisker)
            mask = (out[col] < low) | (out[col] > high)
            n_out = int(mask.sum())
            frac = n_out / max(len(out), 1)

            if mode == "remove" and frac <= max_remove_frac:
                out = out.loc[~mask].copy()
                action = "removed"
            elif mode == "remove" and frac > max_remove_frac:
                out.loc[mask, col] = out.loc[mask, col].clip(lower=low, upper=high)
                action = "capped_due_to_fraction"
            else:  # mode == "cap"
                out[col] = out[col].clip(lower=low, upper=high)
                action = "capped"

            records.append({
                "feature": col, "by": None,
                "low": low, "high": high,
                "n_outliers": n_out, "frac_outliers": frac,
                "action": action
            })
    else:
        # Group-wise IQR per feature (by one or more categoricals)
        keys = [by] if isinstance(by, str) else list(by)
        # Handle NaNs in groups to ensure join works
        for k in keys:
            if out[k].isna().any():
                out[k] = out[k].astype(object).fillna("__MISSING__")

        for col in num_feats:
            g = out.groupby(keys, dropna=False)
            qs = g[col].quantile([0.25, 0.75]).unstack(-1)
            qs.columns = ["q1", "q3"]
            sizes = g.size().rename("n")
            b = qs.join(sizes, how="left")
            b["iqr"]  = b["q3"] - b["q1"]
            b["low"]  = b["q1"] - whisker * b["iqr"]
            b["high"] = b["q3"] + whisker * b["iqr"]

            # Fallback to global bounds for small groups
            low_g, high_g = _iqr_bounds(out[col], whisker)
            small = b["n"] < min_group_size
            b.loc[small, ["low", "high"]] = [low_g, high_g]

            # Join bounds to rows
            out = out.merge(b[["low", "high"]], left_on=keys, right_index=True, how="left")
            mask = (out[col] < out["low"]) | (out[col] > out["high"])
            n_out = int(mask.sum())
            frac = n_out / max(len(out), 1)

            if mode == "remove" and frac <= max_remove_frac:
                out = out.loc[~mask].copy()
                action = "removed"
            elif mode == "remove" and frac > max_remove_frac:
                out.loc[mask, col] = out.loc[mask, col].clip(lower=out.loc[mask, "low"], upper=out.loc[mask, "high"])
                action = "capped_due_to_fraction"
            else:  # mode == "cap"
                out[col] = out[col].clip(lower=out["low"], upper=out["high"])
                action = "capped"

            out = out.drop(columns=["low", "high"])  # cleanup
            records.append({
                "feature": col, "by": "+".join(keys),
                "low": low_g, "high": high_g,  # global fallback (for reference)
                "n_outliers": n_out, "frac_outliers": frac,
                "action": action
            })

    report = pd.DataFrame.from_records(records)
    return out, report.sort_values(["frac_outliers", "feature"], ascending=[False, True]).reset_index(drop=True)

def plot_boxplots(df: pd.DataFrame, num_cols, by: str | None = None, max_cols: int = 6, figsize=(12, 8)):
    """Quick visual check: draws simple matplotlib boxplots (no seaborn)."""
    cols = list(num_cols)[:max_cols]
    n = len(cols)
    ncols = min(3, n)
    nrows = (n + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    axes = np.atleast_1d(axes).ravel()

    for ax, col in zip(axes, cols):
        if by is None:
            ax.boxplot(df[col].dropna(), vert=True, showfliers=True)
            ax.set_title(col)
            ax.set_xticks([])
        else:
            # Show up to top-10 categories by frequency
            topcats = df[by].astype("object").value_counts().head(10).index
            data = [df.loc[df[by] == cat, col].dropna() for cat in topcats]
            ax.boxplot(data, vert=True, showfliers=True)
            ax.set_title(f"{col} by {by}")
            ax.set_xticklabels([str(c) for c in topcats], rotation=45, ha="right")

    # Hide any unused axes
    for j in range(len(cols), len(axes)):
        fig.delaxes(axes[j])

    fig.tight_layout()
    plt.show()


In [ ]:
# 1) Global cleaning (per feature)
train_clean, outlier_report = boxplot_clean(
    train, num_feats,
    mode="remove",        # or "cap"
    whisker=1.5,          # 1.5 (standard), 3.0 (more conservative)
    max_remove_frac=0.10  # auto-cap if >10% would be dropped
)


test_clean, outlier_report_test = boxplot_clean(
    test, num_feats,
    mode="remove",        # or "cap"
    whisker=1.5,          # 1.5 (standard), 3.0 (more conservative)
    max_remove_frac=0.10  # auto-cap if >10% would be dropped
)
# 2) Group-wise cleaning by a categorical (e.g., the most important from your cat_feats)
#    Use "cap" to be gentler on row counts
#'''
#train_clean_by_store, report_store = boxplot_clean(
#    train, num_feats,
#    by="store_id",        # or any one of your cat_feats
#    mode="cap",
#    whisker=1.5
#)
#'''
# 3) Quick visual check (before / after)
plot_boxplots(train, num_feats, max_cols=6)           # before
plot_boxplots(train_clean, num_feats, max_cols=6)     # after

# 4) (Optional) Iterate through a few key categoricals one by one (be cautious: sequential passes get stricter)
# for c in ["store_id", "item_category"]:  # pick 1–2 most relevant cats from cat_feats
#     train, _ = boxplot_clean(train, num_feats, by=c, mode="cap")


# Feature engineering

In [ ]:
def featurize(df):
    df['logical_square_']=df['']**2
    
    df['logical_sqrt_']=np.sqrt(df[''])

    df['logical_mutliplication_']=df['']*df['']

    df['logical_ratio_']=df['']/df['']

    #space for any creative features my dude 
    df['']=







    



    
    
    # end of space my dudeeeeeeee

    #kinda multiplication for categorical(if any ofc)

    df['multi_cat_']=df[''] + '_' + df['']


    #squares for categorical  🤡🤡🤡💀💀💀

    df['square_cat_']=df[''] + '_' + df['']

    return df
    

In [ ]:
train_full=featurize(train)


In [ ]:
test_full=featurize(test)

# skf catboost

## maybe diffirent hyperharameters for catboost?

In [ ]:
model_cfgs=[
    
    dict(name='',
        params=dict()),
    
    dict(name='',
        params=dict()),
    
    dict(name='',
        params=dict()),
    
    dict(name='',
        params=dict()),
    
    dict(name='',
        params=dict()),
    
    
    
]
#find them my dudeeeee

In [ ]:
# for classification:
'''
model_cfgs = [
    # Long-run, small LR, Bayesian bootstrap — a common winner pattern for AUC, esp. S4E7
    dict(name='PS_LongRun_Bayesian_AUC_Imbalanced',
         params=dict(
             loss_function='Logloss',
             eval_metric='AUC',
             iterations=30000,
             learning_rate=0.02,
             depth=8,
             l2_leaf_reg=10.0,
             bootstrap_type='Bayesian',
             bagging_temperature=0.5,
             border_count=254,
             random_strength=1.0,
             use_best_model=True,
             od_type='Iter',
             od_wait=300,
             auto_class_weights='Balanced',
             random_seed=42
         )),

    # Fast, strong CPU default using MVS bootstrap (CatBoost’s CPU default for many cases)
    dict(name='PS_MVS_CPU_Fast',
         params=dict(
             loss_function='Logloss',
             eval_metric='AUC',
             iterations=8000,
             learning_rate=0.05,
             depth=6,
             l2_leaf_reg=8.0,
             bootstrap_type='MVS',
             subsample=0.8,          # active for MVS
             # mvs_reg can be tuned; omit to use auto
             random_strength=2.0,
             border_count=254,
             use_best_model=True,
             od_type='Iter',
             od_wait=200,
             auto_class_weights='Balanced',
             random_seed=42
         )),

    # Robust Bernoulli (SGB-style) bagging; good regularization, stable across folds
    dict(name='PS_Bernoulli_SGB_Robust',
         params=dict(
             loss_function='Logloss',
             eval_metric='AUC',
             iterations=15000,
             learning_rate=0.03,
             depth=8,
             l2_leaf_reg=12.0,
             bootstrap_type='Bernoulli',
             subsample=0.66,         # classic SGB rate
             random_strength=1.5,
             border_count=254,
             use_best_model=True,
             od_type='Iter',
             od_wait=300,
             auto_class_weights='Balanced',
             random_seed=42
         )),

    # GPU-heavy: Poisson bootstrap + Lossguide grow policy for wide/high-card data
    dict(name='PS_GPU_Poisson_Lossguide_BigData',
         params=dict(
             loss_function='Logloss',
             eval_metric='AUC',
             task_type='GPU',
             iterations=20000,
             learning_rate=0.03,
             grow_policy='Lossguide',
             max_leaves=63,          # keep <=64 for speed/quality balance
             min_data_in_leaf=20,
             bootstrap_type='Poisson',  # GPU-only
             subsample=0.66,
             border_count=128,       # 128 often best balance on GPU
             l2_leaf_reg=6.0,
             random_strength=1.0,
             use_best_model=True,
             od_type='Iter',
             od_wait=200,
             auto_class_weights='Balanced',
             random_seed=42
         )),

    # Newton leaf updates (second-order) + mild Bayesian bagging — strong on many tabular AUC tasks
    dict(name='PS_Newton_LeafEstimation_StrongReg',
         params=dict(
             loss_function='Logloss',
             eval_metric='AUC',
             iterations=15000,
             learning_rate=0.03,
             depth=10,
             l2_leaf_reg=12.0,
             leaf_estimation_method='Newton',
             leaf_estimation_iterations=10,
             bootstrap_type='Bayesian',
             bagging_temperature=0.25,
             border_count=254,
             random_strength=1.0,
             use_best_model=True,
             od_type='Iter',
             od_wait=300,
             auto_class_weights='Balanced',
             random_seed=42
         )),
]
'''




# for regression:

'''
model_cfgs = [
    # 1) Long-run, tiny LR, Bayesian bagging — great default for RMSE-driven tabular
    dict(name='PS_Reg_LongRun_Bayesian_RMSE',
         params=dict(
             loss_function='RMSE',
             eval_metric='RMSE',
             iterations=30000,
             learning_rate=0.02,
             depth=8,
             l2_leaf_reg=10.0,
             bootstrap_type='Bayesian',
             bagging_temperature=0.5,
             border_count=254,     # CPU: finer bins
             rsm=0.8,              # CPU only
             random_strength=1.0,
             early_stopping_rounds=300,
             use_best_model=True,
             random_seed=42
         )),

    # 2) Fast, strong CPU setup using MVS bootstrap (good when you want speed)
    dict(name='PS_Reg_MVS_CPU_Fast',
         params=dict(
             loss_function='RMSE',
             eval_metric='RMSE',
             iterations=8000,
             learning_rate=0.05,
             depth=6,
             l2_leaf_reg=8.0,
             bootstrap_type='MVS',
             subsample=0.8,        # active with MVS
             rsm=0.9,              # CPU only
             random_strength=1.5,
             border_count=254,
             early_stopping_rounds=200,
             use_best_model=True,
             random_seed=42
         )),

    # 3) Robust to outliers using Huber loss (heavy-tailed targets)
    dict(name='PS_Reg_Huber_Outliers',
         params=dict(
             loss_function='Huber:delta=1.0',
             eval_metric='RMSE',
             iterations=20000,
             learning_rate=0.03,
             depth=8,
             l2_leaf_reg=12.0,
             bootstrap_type='Bernoulli',
             subsample=0.66,
             border_count=254,
             random_strength=1.0,
             early_stopping_rounds=300,
             use_best_model=True,
             random_seed=42
         )),

    # 4) Insurance/zero-inflated style targets (Tweedie GLM-like objective)
    dict(name='PS_Reg_Tweedie_Exposure',
         params=dict(
             loss_function='Tweedie:variance_power=1.5',  # tune 1.2–1.8
             eval_metric='RMSE',
             iterations=20000,
             learning_rate=0.03,
             depth=8,
             l2_leaf_reg=10.0,
             bootstrap_type='Bayesian',
             bagging_temperature=0.25,
             border_count=254,
             rsm=0.8,              # CPU only
             random_strength=1.0,
             early_stopping_rounds=300,
             use_best_model=True,
             random_seed=42
         )),

    # 5) Wide/high-card data on GPU — Lossguide + Poisson bootstrap
    dict(name='PS_Reg_GPU_Lossguide_BigData',
         params=dict(
             task_type='GPU',
             loss_function='RMSE',
             eval_metric='RMSE',
             iterations=20000,
             learning_rate=0.03,
             grow_policy='Lossguide',
             max_leaves=63,        # keep ≤64 for speed/quality
             min_data_in_leaf=20,
             bootstrap_type='Poisson',  # GPU-only
             subsample=0.66,
             border_count=128,     # GPU bin count
             l2_leaf_reg=6.0,
             random_strength=1.0,
             early_stopping_rounds=200,
             use_best_model=True,
             random_seed=42
         )),
]


'''

In [ ]:
X=train_full.drop('target')
y=train_full['target']

In [ ]:
cat_feats_full=train_full.select_dtypes(include='object').columns.tolist()

## Для Classifier добавить перебор трешхолда(predict_proba)

In [ ]:
skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=seed) #Or just KFold, one word is the diffirence 
n_models=len(model_cfgs)
n_folds=5
oof_preds=np.zeros((len(y), n_models), dtype=float)
test_preds=np.zeros(len(test_full), n_models), dtype=float)

cv_scores=[]

test_pool=Pool(
    data=test_full
)

for m_idx, cfg in enumerate(model_cfgs):
    name, params=cfg['name'], cfg['params']
    oof_pred=np.zeros(len(y), dtype=float)
    test_pred=np.zeros(len(test_full), dtype=float)
    fold_metric=[]
    for fold, (train_idx, eval_idx) in enumerate(skf.split(X,y), 1):
        X_train=X.iloc[train_idx]
        X_eval=X.iloc[eval_idx]
        y_train=y.iloc[train_idx]
        y_eval=y.iloc[eval_idx]

        train_pool=Pool(
            data=X_train,
            label=y_train,
            cat_features=cat_feats_full,
        
        )
        eval_pool=Pool(
            data=X_eval,
            label=y_eval,
            cat_features=cat_feats_full,
        
        )
        model=CatBoostClassifier(        # ну или Regressor
            **params
        )
        model.fit(train_pool, eval_set=eval_pool, verbose=200, use_best_model=True, early_stopping_rounds=400)
        eval_pred=model.predict(eval_pool)
        oof_pred[eval_idx]=eval_pred
        fold_metric.append((y_eval, eval_pred))

        test_pred+=model.predict(test_pool)/n_folds
    print(f'CV :{np.mean(fold_metric):.5f} +- {np.std(fold_metric):.5f}')
    oof_preds[:, m_idx]=oof_pred
    test_preds[:, m_idx]=test_pred
    model_cv=(np.mean(fold_metric), np.std(fold_metric))
    cv_scores.append(name, *model_cv)
    print(f'CV :{model_cv[0]:.5f} ± {model_cv[1]:.5f}')
    

# Meta models

In [ ]:
colnames=[cfg['name'] for cfg in model_cfgs]
oof_df=pd.DataFrame(oof_preds, columns=colnames)
test_df=pd.DataFrame(test_preds, columns=colnames)


## Linreg over predictions(meta model)

In [ ]:
model_lasso=Lasso(alpha=1)

In [ ]:
model_lasso.fit(oof_df, y)

linreg_test_pred = model_lasso.predict(test_df)

In [ ]:
sub1=pd.DataFrame({
    'id': sample['id'],
    'target' : linreg_test_pred
})

In [ ]:
sub1.to_csv('sub_lasso_as_meta_model_first_try.csv', index=False)

## Catboost over predictions(meta model)

In [ ]:
model_cat=CatBoostClassifier( # or Regressor
    iterations=500,
    depth=3,
    learning_rate=0.05,
    l2_leaf_reg=10.0,
    task_type='GPU',
    devices='0',
   # loss_function='RMSE',
)

In [ ]:
model_cat.fit(oof_df, y, verbose=False)

In [ ]:
catboost_test_pred=model_cat.predict(test_df)


In [ ]:
sub2=pd.DataFrame({
    'id':sample['id'],
    'target': catboost_test_pred
})

In [ ]:
sub2.to_csv('catboost_as_meta_model_first_try_gol.csv', index=False)

# Just average of skf catboost

In [ ]:
blend = test_df.mean(axis=1)

In [ ]:
sub3=pd.DataFrame({
    'id': sample['id'],
    'target': blend
})

In [ ]:
sub3.to_csv('just_a_blend_oout_of_skf.csv', index=False)

# optuna for better ensemble МЕТРИКУ ПОМЕНЯТЬ 

In [ ]:
def objective(trial):
     raw = np.array([trial.suggest_float(f"w{i}", 0.0, 1.0) for i in range(n_models)])
     w = raw / (raw.sum() + 1e-12)
     blend = oof_df.values @ w
     return mean_squared_error(y, blend, squared=False) 

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)
best_raw = np.array([study.best_params[f"w{i}"] for i in range(n_models)])
w = best_raw / (best_raw.sum() + 1e-12)
print("Best weights:", dict(zip(colnames, w.round(4))))
test_blend = test_df.values @ w

#Своровал но понял что происходит, надо самим реализовать в след ноутбуке

In [ ]:
sub4=pd.DataFrame({
    'id': sample['id'],
    'target': test_blend
})

In [ ]:
sub4.to_csv('ensemble_skf_optuna.csv', index=False)

# Average of all submissions(or optuna of all submissions, idk) or both my dude 

In [ ]:
mean_pred = pd.concat([
    pd.Series(catboost_test_pred, index=test_df.index, name='catboost'),
    pd.Series(linreg_test_pred,   index=test_df.index, name='linreg'),
    pd.Series(blend,              index=test_df.index, name='blend'),
    pd.Series(test_blend,         index=test_df.index, name='optuna_blend'),
], axis=1).mean(axis=1)

In [ ]:
sub5=pd.DataFrame({
    'id': sample['id'],
    'target':mean_pred
    
})

In [ ]:
sub5.to_csv('mean_of_all_subs_my_dude.csv', index=False)

# МЕТРИКУ ПОМЕНЯТЬ  А ЕЩЕ ЛИНРЕГ ДОБАВИТЬ 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import optuna

# --- 1) Собираем пары (OOF, TEST) для трёх источников ---
# Предполагаем, что у тебя уже есть:
# y               : pandas Series длины N (таргет train)
# oof_df, test_df : базовые OOF/TEST предсказания (DataFrame)
# w               : веса базового blend из Optuna (по столбцам oof_df/test_df)
# blend           : простой средний по колонкам test_df (Series)
# test_blend      : взвешенный по w тестовый blend (ndarray/Series)
# catboost_oof_pred, catboost_test_pred : OOF/TEST из CatBoost-модели

# OOF для mean и opt_w:
oof_mean = oof_df.mean(axis=1).values                    # shape (N,)
oof_opt  = (oof_df.values @ w).ravel()                   # shape (N,)

# Приводим TEST-компоненты к Series с единым индексом (как у blend/test_df)
test_index = getattr(blend, 'index', None) or getattr(test_df, 'index', None)
catboost_test_s = pd.Series(np.asarray(catboost_test_pred).ravel(), index=test_index, name='catboost')
#linreg_test_s = pd.Series(np.asarray(linreg_test_pred).ravel(), index=test_index, name='lasso')

blend_s         = blend.rename('mean') if hasattr(blend, 'rename') else pd.Series(np.asarray(blend).ravel(), index=test_index, name='mean')
test_blend_s    = pd.Series(np.asarray(test_blend).ravel(), index=test_index, name='opt_w')

# Проверки форм и длин
assert len(y) == len(oof_mean) == len(oof_opt) == len(catboost_oof_pred)
assert len(catboost_test_s) == len(blend_s) == len(test_blend_s)

# Стек OOF (N x 3) и TEST (T x 3)
oof_stack  = np.column_stack([
    np.asarray(catboost_oof_pred).ravel(),  # catboost
    oof_mean,                               # mean
    oof_opt,                                 # opt_w (базовый взвешенный)
    #np.asarray(linreg_oof_pred).ravel()
])
test_stack = np.column_stack([
    catboost_test_s.values,
    blend_s.values,
    test_blend_s.values,
   # linreg_test_s.values,
])
names = ['catboost', 'mean', 'opt_w']

# --- 2) Оптимизация весов по OOF через Optuna ---
def objective(trial):
    raw = np.array([trial.suggest_float(f"w_{n}", 0.0, 1.0) for n in names], dtype=float)
    w2  = raw / (raw.sum() + 1e-12)                 # simplex
    blend_oof = oof_stack @ w2
    rmse = mean_squared_error(y, blend_oof, squared=False)
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=300, show_progress_bar=False)

best_raw = np.array([study.best_params[f"w_{n}"] for n in names], dtype=float)
w2 = best_raw / (best_raw.sum() + 1e-12)
print("Best weights (catboost, mean, opt_w):", dict(zip(names, np.round(w2, 4))))

# --- 3) Финальный TEST-бленд из трёх источников ---
final_test_pred = test_stack @ w2  # shape (T,)



In [ ]:
sub6=pd.DataFrame({
    'id': sample['id'],
    'target': final_test_pred
})

In [ ]:
sub6.to_csv('optuna_of_all_my_subs_first_try.csv', index=False)